In [ ]:
! git clone https://github.com/Yang-Hyun-Jun/RLPortfolio-PPO-COLAB.git

In [ ]:
! pip install mplfinance

In [3]:
import sys
import os
import torch

In [4]:
sys.path.insert(0, "/content/RLPortfolio-PPO-COLAB")

In [5]:
os.makedirs("/content/Metrics", exist_ok=True)
os.makedirs("/content/Models", exist_ok=True)

In [6]:
import DataManager
import utils
import Learner
import ReplayMemory
import Test
import Agent
import Network 

from Learner import PPOLearner
from Test import PPOTester

In [7]:
# Random seed 

seed = 8
Learner.seed = seed
ReplayMemory.seed = seed
Test.seed = seed
Agent.seed = seed
Network.seed = seed

In [ ]:
# Data Load

stock_list = ["BIDU", "TCOM", "AAPL", "COST"]
path_list = []

for stock_name in stock_list:
  path = utils.SAVE_DIR + "/RLPortfolio-PPO-COLAB/Data/" + stock_name
  path_list.append(path)

train_data, test_data = DataManager.get_data_tensor(path_list,
                                                    train_date_start="2014-06-02",
                                                    train_date_end="2019-12-31",
                                                    test_date_start="2020-01-02",
                                                    test_date_end="2021-12-31")

In [10]:
""""
K: 거래 종목 수
lr: learing rate (default 1e-5)
tau: soft target update (default 0.005)
delta: hold action threshold (default 0.005)
discount_factor: gamma (default 0.9)
num_episode: episode (default 50)
balance: 초기 투자금 (default 12000 달러) 
batch_size: 배치 사이즈 (default 30)
memory_size: 버퍼 사이즈 (default 100)
min_trading_price: 최소 거래 금액 (default 0달러)
max_trading_price: 최대 거래 금액 (default 400달러) 
repre: 테스트시 대푯값 ("mean" or "mode")
"""

# Hyperparameters
K = len(stock_list)
lr = 1e-6
tau = 0.005
delta = 0.005
discount_factor = 0.9
num_episode = 30
balance = 12000
batch_size = 20
memory_size = 100
min_trading_price = 0
max_trading_price = 400

# paramter1 for training
parameters1 = {"lr":lr, 
               "tau":tau,
               "K":K,
               "delta":delta, "cost":0.0,
               "chart_data":train_data,
               "discount_factor":discount_factor,
               "min_trading_price":min_trading_price,
               "max_trading_price":max_trading_price,
               "batch_size":batch_size,
               "memory_size":memory_size}

# parameter2 : for test
parameters2 = {"K":K,
               "delta":delta,
               "balance":balance, "cost":0.0025,
               "min_trading_price":min_trading_price,
               "max_trading_price":max_trading_price,
               "test_data":test_data,
               "holding":False} 

In [ ]:
# 학습 수행
learner = PPOLearner(**parameters1)
learner.run(num_episode=num_episode, balance=balance)
learner.save_model(critic_path=utils.SAVE_DIR + "/Models" + "/PPOPortfolio_critic.pth",
                   actor_path=utils.SAVE_DIR + "/Models" + "/PPOPortfolio_actor.pth",
                   score_net_path=utils.SAVE_DIR + "/Models" + "/PPOPortfolio_score.pth")

In [ ]:
# 테스트 수행
tester = PPOTester(**parameters2)
tester.run()

In [ ]:
! zip -r /content/Metrics.zip /content/Metrics

In [ ]:
! zip -r /content/Models.zip /content/Models

In [ ]:
from google.colab import files
files.download("/content/Metrics.zip")
files.download("/content/Models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>